# Hands on introduction to ancient microbiome analysis
Author: Maxime Borry  
Date: 12/08/2021

In this tutorial, we're going to go through the steps necessary to:
- generate a taxonomic profile table with [metaphlan v3.0](https://github.com/biobakery/MetaPhlAn)
- have a look at metaphlan results with [Pavian](https://github.com/fbreitwieser/pavian) and generate a [Krona plot](https://github.com/marbl/Krona/wiki)
- Compare the alpha diversity of our samples vs the diversity of modern human gut samples
- Compare the composition of our samples vs modern gut samples, and see where they fit in a lower dimensional space

## 0. Quick intro to Jupyter Notebooks

This a markdown cell

In [12]:
print("This is a python cell")

This is a python cell


In [13]:
! echo "This is how to run a single line bash command"

This is how to run a single line bash command


In [15]:
%%bash
echo "This how to run a multi"
echo "lines bash command"

This how to run a multi
lines bash command


## 1. Data pre-processing

Before starting to analyze our data, we will need to pre-process them to remove reads mapping to the host genome, here, *Homo sapiens*

To do so, I've used [nf-core/eager](https://github.com/nf-core/eager)

I've already pre-processed the data, and the resulting cleaned files are available in the [data/eager_cleaned](data/eager_cleaned), but the basic eager command to do so is:

` nextflow run nf-core/eager -profile <docker/singularity/podman/conda/institute> --input '*_R{1,2}.fastq.gz' --fasta 'human_genome.fasta' --hostremoval_input_fastq`

## 2. Adapter sequence trimming and low-quality bases trimming

Sequencing adapters are small DNA sequences adding prior to DNA sequencing to allow the DNA fragments to attach to the sequencing flow cells. Because these adapters could interfere with downtstream analyses, we need to remove them before proceeding any further.
Furthermore, because the quality of the sequencing is not always optimal, we need to remove bases of lower sequencing quality to might lead to spurious results in downstream analyses.

To perform both of these tasks, we'll use the [AdapterRemoval](https://github.com/MikkelSchubert/adapterremoval) program.

In [16]:
! AdapterRemoval -h

AdapterRemoval ver. 2.3.2

This program searches for and removes remnant adapter sequences from
your read data.  The program can analyze both single end and paired end
data.  For detailed explanation of the parameters, please refer to the
man page.  For comments, suggestions  and feedback please contact Stinus
Lindgreen (stinus@binf.ku.dk) and Mikkel Schubert (MikkelSch@gmail.com).

If you use the program, please cite the paper:
    Schubert, Lindgreen, and Orlando (2016). AdapterRemoval v2: rapid
    adapter trimming, identification, and read merging.
    BMC Research Notes, 12;9(1):88.

    http://bmcresnotes.biomedcentral.com/articles/10.1186/s13104-016-1900-2


OPTIONS:
    --help  
        Display this message.

    --version  
        Print the version string.


    --file1 FILE [FILE ...]
        Input files containing mate 1 reads or single-ended reads; one or
        more files may be listed [REQUIRED].

    --file2 [FILE ...]
        Input files containing mate 2 reads; if us

In [29]:
%%bash
AdapterRemoval --file1 ../data/eager_cleaned/ZSM028_R1.fastq.gz \
    --file2 ../data/eager_cleaned/ZSM028_R2.fastq.gz \
    --basename ../results/adapterremoval/ZSM028 \
    --gzip \
    --minquality 20 \
    --trimqualities \
    --collapse

Trimming paired end reads ...
Opening FASTQ file '../data/eager_cleaned/ZSM028_R1.fastq.gz', line numbers start at 1
Opening FASTQ file '../data/eager_cleaned/ZSM028_R2.fastq.gz', line numbers start at 1
Processed a total of 1,996,618 reads in 25.6s; 78,000 reads per second on average ...


Since Metaphlan doesn't use the paired-end information, we'll combine all output files together

In [31]:
! cat ../results/adapterremoval/*.collapsed.* ../results/adapterremoval/*.truncated.* ../results/adapterremoval/*.singleton.*  > ../results/adapterremoval/ZSM028.combined.fastq.gz

## 3. Taxonomic profiling with Metaphlan

In [20]:
! metaphlan  --help

usage: metaphlan --input_type {fastq,fasta,bowtie2out,sam} [--force]
                 [--bowtie2db METAPHLAN_BOWTIE2_DB] [-x INDEX]
                 [--bt2_ps BowTie2 presets] [--bowtie2_exe BOWTIE2_EXE]
                 [--bowtie2_build BOWTIE2_BUILD] [--bowtie2out FILE_NAME]
                 [--min_mapq_val MIN_MAPQ_VAL] [--no_map] [--tmp_dir]
                 [--tax_lev TAXONOMIC_LEVEL] [--min_cu_len]
                 [--min_alignment_len] [--add_viruses] [--ignore_eukaryotes]
                 [--ignore_bacteria] [--ignore_archaea] [--stat_q]
                 [--perc_nonzero] [--ignore_markers IGNORE_MARKERS]
                 [--avoid_disqm] [--stat] [-t ANALYSIS TYPE]
                 [--nreads NUMBER_OF_READS] [--pres_th PRESENCE_THRESHOLD]
                 [--clade] [--min_ab] [-o output file] [--sample_id_key name]
                 [--use_group_representative] [--sample_id value]
                 [-s sam_output_file] [--legacy-output] [--CAMI_format_output]
                 [--u

In [46]:
%%bash
metaphlan ../results/adapterremoval/ZSM028.combined.fastq.gz  \
    --input_type fastq \
    --bowtie2db /home/maxime_borry/.conda/envs/summer_school_microbiome/lib/python3.9/site-packages/metaphlan/metaphlan_databases/  \
    --bowtie2out ../results/metaphlan/ZSM028.bt2.out  \
    > ../results/metaphlan/ZSM028.metaphlan_profile.txt

An additional column listing the merged species is added to the MetaPhlAn output.


The main results files that we're interested in is located at [../results/metaphlan/ZSM028.metaphlan_profile.txt](../results/metaphlan/ZSM028.metaphlan_profile.txt)

## 4. Visualizing metaphlan results with Pavian

[Pavian](https://github.com/fbreitwieser/pavian) is an interactive app to explore results of different taxonomic classifiers

There are different ways to run it:
- If you have [docker](https://www.docker.com/) installed (and are somehow familiar with it)

```bash
docker pull 'florianbw/pavian'
docker run --rm -p 5000:80 florianbw/pavian
```

Then open your browser and visit [localhost:5000](localhost:5000)

- If you are familiar with [R](https://www.r-project.org/)

```r
if (!require(remotes)) { install.packages("remotes") }
remotes::install_github("fbreitwieser/pavian")

pavian::runApp(port=5000)
```

Then open your browser and visit [localhost:5000](localhost:5000)

- Otherwise, just visit [fbreitwieser.shinyapps.io/pavian](https://fbreitwieser.shinyapps.io/pavian/.).